<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install knmy
!pip install knmy

     |████████████████████████████████| 6.4MB 4.7MB/s 
  Created wheel for knmy: filename=knmy-1.0.0-py2.py3-none-any.whl size=5501 sha256=8d5a5d3c1a15105351c327bf82ada788f663871d295162f9ddb71b9eb0052f59
  Stored in directory: /root/.cache/pip/wheels/a1/15/f8/75d11ed17dbbb170e1a83741e2d651f6655d2be905d3ed5616
Successfully built knmy


In [2]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
from knmy import knmy
import pandas as pd
import numpy as np

def knmi_get(start, end, stations=[240, 260]):
    
    # knmy.get_hourly_data returns a tuple with 4 items. Immediately index to [3] to get the df with weather variables. 
    knmi_data = knmy.get_hourly_data(stations=[240], start=start, end=end,
                            inseason=False, variables=['ALL'], parse=True)[3]
    
    # Rename columns
    cols = ['weatherstation', 'date', 'hour', 'winddirection', 'windspeed_avg', \
            'windspeed_10m','windspeed_max', 'temperature', 'temperature_min', \
            'temperature_dewpoint', 'sunduration', 'sunradiation', \
            'precipitationduration', 'precipitation', 'airpressure', \
            'horizontalview', 'cloudcover', 'relativehumidity', 'weathercode', \
            'weathercodeindicator', 'mist', 'rain', 'snow', 'storm', 'ice']
    knmi_data.columns = cols
    
    # Drop useless columns
    knmi_data.drop(['winddirection', 'windspeed_10m', 'temperature_dewpoint', \
                    'horizontalview', 'cloudcover', 'weathercode', \
                    'weathercodeindicator'], axis = 1, inplace=True)
    # Drop first row since it's actually a header, then reset index
    knmi_data.drop([0], inplace=True)
    knmi_data.reset_index(drop=True, inplace=True)
    # Subtract one hour to make data in line with NDW
    knmi_data.hour = knmi_data.hour.astype(int) - 1
    # Remove columns with NA vals
    knmi_data.dropna(axis=1, inplace=True)
    # Make dataframe numeric
    knmi_data = knmi_data.apply(pd.to_numeric)
    # Remove negative precipitation values (-1 means 0.05mm or less)
    knmi_data['precipitation'] = \
        np.where(knmi_data['precipitation'] < 0, 0, knmi_data['precipitation'])
    # Make date col a datetime object
    knmi_data['date'] = pd.to_datetime(knmi_data['date'], yearfirst = True, 
                                                  format = '%Y%m%d').dt.date
    return knmi_data

In [31]:
*_, data = knmy.get_hourly_data(stations = [260], start = 2015010101, end = 2017010624,
                                                             inseason = True, variables = ['WIND', 'TEMP'], parse = True)
data.drop([0], inplace = True)

AttributeError: ignored

In [20]:
data.head()

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10,TD
0,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10,TD
1,260,20150101,1,210,40,40,60,24,NaN,5
2,260,20150101,2,200,40,40,60,22,NaN,-3
3,260,20150101,3,200,40,40,70,16,NaN,-3
4,260,20150101,4,220,40,30,80,13,NaN,-4


In [29]:
data.index = pd.to_datetime(data.YYYYMMDD + data.HH.str.zfill(2), format = "%Y%m%d%H", yearfirst = True)

ValueError: ignored

In [30]:
data.YYYYMMDD + data.HH.str.zfill(2)

0      YYYYMMDDHH
1      2015010101
2      2015010102
3      2015010103
4      2015010104
          ...    
428    2017010620
429    2017010621
430    2017010622
431    2017010623
432    2017010624
Length: 433, dtype: object